# Necessary lib

In [1]:
import json
import spacy
import random
from spacy.training.example import Example
from spacy.scorer import Scorer
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm import tqdm
import json

In [2]:
print(spacy.__version__)

3.8.4


In [3]:
import sys
print(sys.executable)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\python.exe


In [4]:
with open('Corona2.json','r') as f:
    data = json.load(f)

In [5]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [6]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [7]:
data['examples'][0]['annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

# Training data

In [8]:
training_data = []
for example in data['examples']:
    tempt_inf = {}
    tempt_inf['text'] = example['content']
    tempt_inf['entities'] = []
    for annotation in example['annotations']:
        tempt_inf['entities'].append((annotation['start'], annotation['end'], annotation['tag_name'].upper()))
        training_data.append(tempt_inf)

In [9]:
training_data[0]['text'][360:370]

'Diosmectit'

In [10]:
training_data[0]['entities']

[(360, 371, 'MEDICINE'),
 (383, 408, 'MEDICINE'),
 (104, 112, 'MEDICALCONDITION'),
 (679, 689, 'MEDICINE'),
 (6, 23, 'MEDICINE'),
 (25, 37, 'MEDICINE'),
 (461, 470, 'MEDICALCONDITION'),
 (577, 589, 'MEDICINE'),
 (853, 865, 'MEDICALCONDITION'),
 (188, 198, 'MEDICINE'),
 (754, 762, 'MEDICALCONDITION'),
 (870, 880, 'MEDICALCONDITION'),
 (823, 833, 'MEDICINE'),
 (852, 853, 'MEDICALCONDITION'),
 (461, 469, 'MEDICALCONDITION'),
 (535, 543, 'MEDICALCONDITION'),
 (692, 704, 'MEDICINE'),
 (563, 571, 'MEDICALCONDITION')]

In [11]:
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # spacy object to store the examples

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fine tuning

In [12]:
for sample in tqdm(training_data):
    text = sample['text']
    labels = sample['entities']
    doc = nlp.make_doc(text) # create a doc object from the text
    enty = []
    for start, end, label in labels: 
        # create character based spans (start and end index of the span) that are required for NER
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity") # the span is misaligned --> skip 
        else:
            enty.append(span)
    filtered = filter_spans(enty) # filter out overlapping entities
    doc.ents = filtered # set entities in the doc object
    db.add(doc)

db.to_disk("train.spacy") # save the docbin object

 76%|███████▌  | 224/295 [00:00<00:00, 1111.15it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

100%|██████████| 295/295 [00:00<00:00, 1297.09it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [13]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     80.71    1.86    1.05    8.12    0.02
  0     200        482.21   4851.56   90.99   91.26   90.71    0.91
  1     400        206.65    576.70   97.43   96.91   97.96    0.97
  2     600        133.13    218.64   98.40   98.80   98.01    0.98
  2     800        127.30    116.90   98.17   98.62   97.72    0.98
  3    1000        220.66    209.27   98.33   98.24   98.41    0.98
  4    1200        252.77    190.20   98.37   97.93   98.81    0.98
  5    1400        349.22    175.55   98.41   98.41   98.41    0

In [17]:
nlp_med = spacy.load("./output/model-best") # load the best model

In [18]:
# print(nlp_med.pipe_labels["ner"])

# Test

In [27]:
doc = nlp_med("Urinary tract infections are frequently caused by Escherichia coli, especially in individuals with weakened immune systems.")
colors = {"PATHOGEN": "#F9E79F", "MEDICINE": "#82E0AA", "MEDICAL_CONDITION": "#82E0AA"}
options = {"colors:": colors}
# spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

print("Medical NER Output:")
for ent in doc.ents:
    print(f"{ent.text} -> {ent.label_}")

Medical NER Output:
Escherichia coli -> PATHOGEN


# Display by HTML 

In [28]:
from spacy import displacy
html = displacy.render(doc, style="ent", options=options, jupyter=False)
with open("med_output.html", "w", encoding="utf-8") as f:
    f.write(html)